In [1]:
#get authorization from google drive
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#load original record
#import glob
#File_path = "/Users/prajwal/Desktop/Columbia/C4SR/hnyc_cd_processing-master/input/1880/nypl_1880_clean_records.txt"
#with open(file_path) as f:
  #data = f.readlines()  

In [3]:
#load original record
#import glob
#file_path = "/Users/prajwal/Desktop/Columbia/C4SR/doubt/dump/doubt5 filter11.txt"
#with open(file_path) as f:
  #data = f.readlines()  

In [4]:
#data

In [5]:
"""
n = len(data)
full_entry = []
for i in range(n):
  ls = data[i].split()
  if ls:
    if ls[0][1:] != "***":
      full_entry.append(' '.join(ls))
"""

'\nn = len(data)\nfull_entry = []\nfor i in range(n):\n  ls = data[i].split()\n  if ls:\n    if ls[0][1:] != "***":\n      full_entry.append(\' \'.join(ls))\n'

In [6]:
#retrieve result
#import pickle
#with open("/Users/prajwal/Desktop/Columbia/C4SR/hnyc_cd_processing-master/input/1880/prediction.pkl", 'rb') as f:
  #d = pickle.load(f)

In [7]:
#retrieve result
import pickle
#with open("/Users/prajwal/Desktop/Columbia/C4SR/doubt/doubt6-filter11.pkl", 'rb') as f:
#with open('/Users/prajwal/Desktop/Columbia/C4SR/mn_1850/doubt-mn1850-v7.pkl', 'rb') as f:
#with open('/Users/prajwal/Desktop/Columbia/C4SR/tesseract output/run1/mn-1880-3.pkl', 'rb') as f:
with open('/Users/prajwal/Desktop/Columbia/C4SR/tesseract lib/bk 1850/address 2/data2/bk-1850-3-address2.pkl', 'rb') as f:
  d = pickle.load(f)

In [8]:
n = len(d)

In [9]:
n

15676

##Extract different components from prediction results

Only extract information that we need and ignore delimiters.

In [10]:
output = []
for record in d:
    d_record = {}
    try:
        d_record["index"] = int(record[0])
        n = len(record[1])
        start = 1
        for i in range(1, n + 1):
            if i <  n:
              #skip delimiters
              if record[2][i] == "D":
                start = i+1
                continue
              elif record[2][i+1] == "D":
                try:
                  d_record[record[2][i]].append(record[1][(start-1):i])
                except:
                  d_record[record[2][i]] = [record[1][(start-1):i]]
                start = i + 2
            else:
              try:
                  d_record[record[2][i]].append(record[1][(start-1):])
              except:
                  d_record[record[2][i]] = [record[1][(start-1):]]

        output.append(d_record)
    except:
        pass

Further organize prediction results. 

In [11]:
agg = {}
for record in output:
  new = {}

  try:
    occupation = record['OC']
    new['Occupation'] = []
    for oc in occupation:
      new['Occupation'].append(' '.join(oc))
    new['Occupation'] = ' '.join(new['Occupation'])
  except:
    pass

  try:
    #deal with widow marriage status
    new['Name'] = []
    names = record['NC']
    for name in names:
      name_new = name[0]
      n = len(name)
      for i in range(1,n):
        if name[i] == '.':
          name_new += name[i]
        else:
          name_new += ' '
          name_new += name[i]
      new['Name'].append(name_new)
  
  except:
    pass

  try:
    #multiple address
    new['Address'] = []
    addresses = record['AC']
    for address in addresses:
      address_new = address[0]
      n = len(address)
      for i in range(1,n):
        if address[i] == '.':
          address_new += address[i]
        else:
          address_new += ' '
          address_new += address[i]
      new['Address'].append(address_new)

  except:
    pass
  
  agg[str(record['index'])] = new

In [12]:
#n = len(full_entry)
n = len(d)
final_output = {}
#generate fields
#for i in range(1,n+1):
for i in range(0,n):
    cd = {}
    cd['address'] = ''
    #full_entry_ls = full_entry[i-1].split()
    #cd['Index'] = full_entry_ls[0]
    #cd['Full Entry'] = ' '.join(full_entry_ls[1:])
    try:
        record = agg[str(i)]

        try:
            address = record['Address']
            for i in range(len(address)-1):
              h_add = address[i]
              h_add_1 = h_add.split()
              if ((h_add_1[0] in ('h','h.','w','w.','W','W.')) or (h_add_1[0].isdigit()) or (h_add_1[1].isdigit()) or (h_add_1[2].isdigit())):
                continue
              else:
                cd['address'] = ','.join(address[i])

        except:
            pass

        final_output[str(i)] = cd
    except:
        pass

In [13]:
len(final_output)

15676

#Generate final fields

Come up with a list of cities.

In [14]:
cities = ['Hoboken', 'N. J.', 'Spain', 'B’klyn', 'Mt. Vernon','Brooklyn','Jersey City']
def has_city(address, cities = cities):
  for city in cities:
    if city in address:
      return city
  return ''

For address, if there is a house number then further extract street name and city. If not, assume the address is a city name.

Notice that sometimes house number is not always at the beginning of the address. Example: h r 504 W. 55th

In [15]:
#n = len(full_entry)
n = len(d)
final_output = {}
#generate fields
#for i in range(1,n+1):
for i in range(0,n):
  try:
    print(i, agg[str(i)])
  except:
    agg[str(i)] = ''
  cd = {}
  cd['Full Name'] = ''
  cd['First Name'] = ''
  cd['Last Name'] = ''
  cd['Middle Name'] = ''
  cd['Widow-flag'] = 'False'
  cd['Widow Of'] = ''
  cd['Title'] = ''
  cd['Occupation'] = ''
  cd['Full Address'] = ''
  cd['H Status-flag'] = ''
  cd['H Address'] = ''
  cd['H House Number'] = ''
  cd['H Street Name'] = ''
  cd['H City'] = ''
  cd['Address 2'] = ''
  cd['Address 2 House Number'] = ''
  cd['Address 2 Street Name'] = ''
  cd['Address 2 City'] = ''
  #full_entry_ls = full_entry[i-1].split()
  #cd['Index'] = full_entry_ls[0]
  #cd['Full Entry'] = ' '.join(full_entry_ls[1:])

  try:
      record = agg[str(i)]
  
      #name
      try:
        name = record['Name']
        cd['Full Name'] = name[0]
        name_ls = name[0].split()
        if len(name_ls) == 2:
          cd['Last Name'] = name_ls[0]
          cd['First Name'] = name_ls[1]
        elif len(name_ls) == 3:
          cd['Last Name'] = name_ls[0]
          cd['First Name'] = name_ls[1]
          cd['Middle Name'] = name_ls[2]

        #widow flag
        if len(name) == 2:
          name_ls2 = name[1].split()
          if name_ls2[0] == 'widow' or 'wid.':
            cd['Widow-flag'] = 'True'
          if len(name_ls2) > 1:
            cd['Widow Of'] = ' '.join(name_ls2[1:])
      except:
        pass

      #occupation
      try:
        cd['Occupation'] = record['Occupation']
      except:
        pass

      #address
      try:
        address = record['Address']
        cd['Full Address'] = ','.join(address)
        if len(address) == 1:
          cd['H Address'] = address[0]
          address_1 = address[0].split()
          if ((address_1[0] == 'h') or (address_1[0] == 'h.')):
            cd['H Status-flag'] = 'True'
            start = 1
          else:
            cd['H Status-flag'] = 'Assume'
            start = 0

          house_num = False
          if address_1[start].isdigit():
            house_num = True
          elif address_1[start+1].isdigit():
            house_num = True
            start += 1

          if house_num:
            cd['H House Number'] = address_1[start]
            #city
            city = has_city(cities, address_1)
            if city:
              cd['H City'] = city
              cd['H Street Name'] = ' '.join(address_1[(start+1):]).replace(city,'').strip()
            else:
              cd['H Street Name'] = ' '.join(address_1[(start+1):])
          else:
            cd['H City'] = ' '.join(address_1[start:])

        else:
          add = address[0]
          add_ls = add.split()
          cd['Address 2'] = add
          start = 0
          house_num = False
          if add_ls[start].isdigit():
            house_num = True
          elif add_ls[start+1].isdigit():
            house_num = True
            start += 1
          if house_num:
            cd['Address 2 House Number'] = add_ls[start]
            #city
            city = has_city(add)
            if city:
              cd['Address 2 City'] = city
              cd['Address 2 Street Name'] = ' '.join(add_ls[(start+1):]).replace(city, '').strip()
            else:
              cd['Address 2 Street Name'] = ' '.join(add_ls[(start+1):])
          else:
            cd['Address 2 City'] = ' '.join(add_ls[start:])

          add_1 = address[1]
          add_ls = add_1.split()
          if ((add_ls[0] == 'h') or (add_ls[0] == 'h.')):
            cd['H Address'] = add_1
            cd['H Status-flag'] = 'True'
            start = 1
            house_num = False
            if add_ls[start].isdigit():
              house_num = True
            elif add_ls[start+1].isdigit():
              house_num = True
              start += 1
            if house_num:
              cd['H House Number'] = add_ls[start]
              #city
              city = has_city(add_1)
              if city:
                cd['H City'] = city
                cd['H Street Name'] = ' '.join(add_ls[(start+1):]).replace(city, '').strip()
              else:
                cd['H Street Name'] = ' '.join(add_ls[(start+1):])
            else:
              cd['H City'] = ' '.join(add_ls[start:])

      except:
        pass

  except:
      #break
      continue

  final_output[str(i)] = cd

0 {'Name': [], 'Address': []}
1 {'Name': [], 'Address': []}
2 {'Name': [], 'Address': []}
3 {'Name': [], 'Address': []}
4 {'Name': [], 'Address': ['merchant']}
5 {'Name': [], 'Address': []}
6 {'Name': [], 'Address': ['Court']}
7 {'Name': [], 'Address': ['141 Front']}
8 {'Name': [], 'Address': []}
9 {'Name': [], 'Address': []}
10 {'Name': [], 'Address': ['2954 Pearl']}
11 {'Name': [], 'Address': []}
12 {'Name': [], 'Address': ['Myrtle c Jay']}
13 {'Name': [], 'Address': []}
14 {'Name': [], 'Address': []}
15 {'Name': [], 'Address': []}
16 {'Name': [], 'Address': []}
17 {'Name': [], 'Address': []}
18 {'Name': [], 'Address': []}
19 {'Name': ['Y'], 'Address': []}
20 {'Name': [], 'Address': []}
21 {'Name': [], 'Address': []}
22 {'Name': [], 'Address': []}
23 {'Name': [], 'Address': []}
24 {'Name': [], 'Address': ['73 William']}
25 {'Name': [], 'Address': ['15 Cortlandt']}
26 {'Name': [], 'Address': []}
27 {'Name': [], 'Address': ['23 Myrtle av']}
28 {'Name': [], 'Address': []}
29 {'Name': []

947 {'Name': [], 'Address': ['42 Water']}
948 {'Name': [], 'Address': []}
949 {'Name': [], 'Address': ['38 South']}
950 {'Name': [], 'Address': []}
951 {'Name': [], 'Address': []}
952 {'Name': [], 'Address': []}
953 {'Name': [], 'Address': ['38 South']}
954 {'Name': [], 'Address': []}
955 {'Name': [], 'Address': []}
956 {'Name': [], 'Address': []}
957 {'Name': [], 'Address': ['Hoyt']}
958 {'Name': [], 'Address': ['105 Wall']}
959 {'Name': [], 'Address': []}
960 {'Name': [], 'Address': []}
961 {'Name': [], 'Address': []}
962 {'Name': [], 'Address': []}
963 {'Name': [], 'Address': []}
964 {'Name': [], 'Address': []}
965 {'Name': [], 'Address': []}
966 {'Name': [], 'Address': []}
967 {'Name': [], 'Address': []}
968 {'Name': [], 'Address': []}
969 {'Name': [], 'Address': []}
970 {'Name': [], 'Address': []}
971 {'Name': [], 'Address': []}
972 {'Name': [], 'Address': ['57 Water']}
973 {'Name': [], 'Address': ['Fulton c Furman']}
974 {'Name': [], 'Address': []}
975 {'Name': [], 'Address': []}

1697 {'Name': [], 'Address': []}
1698 {'Name': [], 'Address': []}
1699 {'Name': [], 'Address': []}
1700 {'Name': [], 'Address': []}
1701 {'Name': [], 'Address': []}
1702 {'Name': [], 'Address': []}
1703 {'Name': [], 'Address': []}
1704 {'Name': [], 'Address': []}
1705 {'Name': [], 'Address': []}
1706 {'Name': [], 'Address': []}
1707 {'Name': [], 'Address': []}
1708 {'Name': [], 'Address': []}
1709 {'Name': [], 'Address': []}
1710 {'Name': [], 'Address': []}
1711 {'Name': [], 'Address': []}
1712 {'Name': [], 'Address': []}
1713 {'Name': [], 'Address': []}
1714 {'Name': [], 'Address': []}
1715 {'Name': [], 'Address': []}
1716 {'Name': [], 'Address': []}
1717 {'Name': [], 'Address': []}
1718 {'Name': [], 'Address': []}
1719 {'Name': [], 'Address': []}
1720 {'Name': [], 'Address': []}
1721 {'Name': [], 'Address': []}
1722 {'Name': [], 'Address': []}
1723 {'Name': [], 'Address': []}
1724 {'Name': [], 'Address': []}
1725 {'Name': [], 'Address': []}
1726 {'Name': [], 'Address': []}
1727 {'Nam

2447 {'Name': [], 'Address': []}
2448 {'Name': [], 'Address': []}
2449 {'Name': [], 'Address': []}
2450 {'Name': [], 'Address': []}
2451 {'Name': [], 'Address': ['35 Ferry']}
2452 {'Name': [], 'Address': ['Bedford av n Flushing av']}
2453 {'Name': [], 'Address': ['mer']}
2454 {'Name': [], 'Address': []}
2455 {'Name': [], 'Address': []}
2456 {'Name': [], 'Address': []}
2457 {'Name': [], 'Address': []}
2458 {'Name': [], 'Address': []}
2459 {'Name': [], 'Address': []}
2460 {'Name': [], 'Address': []}
2461 {'Name': [], 'Address': []}
2462 {'Name': [], 'Address': []}
2463 {'Name': [], 'Address': []}
2464 {'Name': [], 'Address': []}
2465 {'Name': [], 'Address': []}
2466 {'Name': [], 'Address': []}
2467 {'Name': [], 'Address': []}
2468 {'Name': [], 'Address': []}
2469 {'Name': [], 'Address': []}
2470 {'Name': [], 'Address': []}
2471 {'Name': [], 'Address': []}
2472 {'Name': [], 'Address': []}
2473 {'Name': [], 'Address': []}
2474 {'Name': [], 'Address': []}
2475 {'Name': [], 'Address': []}
24

3258 {'Name': [], 'Address': []}
3259 {'Name': [], 'Address': []}
3260 {'Name': [], 'Address': []}
3261 {'Name': [], 'Address': []}
3262 {'Name': [], 'Address': []}
3263 {'Name': [], 'Address': []}
3264 {'Name': [], 'Address': []}
3265 {'Name': [], 'Address': []}
3266 {'Name': [], 'Address': []}
3267 {'Name': [], 'Address': []}
3268 {'Name': [], 'Address': []}
3269 {'Name': [], 'Address': []}
3270 {'Name': [], 'Address': []}
3271 {'Name': [], 'Address': []}
3272 {'Name': [], 'Address': []}
3273 {'Name': [], 'Address': []}
3274 {'Name': [], 'Address': []}
3275 {'Name': [], 'Address': []}
3276 {'Name': [], 'Address': []}
3277 {'Name': [], 'Address': []}
3278 {'Name': [], 'Address': []}
3279 {'Name': [], 'Address': []}
3280 {'Name': [], 'Address': []}
3281 {'Name': [], 'Address': []}
3282 {'Name': [], 'Address': []}
3283 {'Name': [], 'Address': []}
3284 {'Name': [], 'Address': ['reft y 19 Fulton market']}
3285 {'Name': [], 'Address': []}
3286 {'Name': [], 'Address': []}
3287 {'Name': [], 

...

11444 {'Name': [], 'Address': []}
11445 {'Name': [], 'Address': []}
11446 {'Name': [], 'Address': []}
11447 {'Name': [], 'Address': []}
11448 {'Name': [], 'Address': []}
11449 {'Name': [], 'Address': []}
11450 {'Name': [], 'Address': []}
11451 {'Name': [], 'Address': []}
11452 {'Name': [], 'Address': []}
11453 {'Name': [], 'Address': []}
11454 {'Name': [], 'Address': []}
11455 {'Name': [], 'Address': []}
11456 {'Name': [], 'Address': ['ink manuf r 5 Spruce']}
11457 {'Name': [], 'Address': []}
11458 {'Name': [], 'Address': ['47 Cedar']}
11459 {'Name': [], 'Address': []}
11460 {'Name': [], 'Address': ['Fulton market']}
11461 {'Name': [], 'Address': []}
11462 {'Name': [], 'Address': []}
11463 {'Name': [], 'Address': []}
11464 {'Name': [], 'Address': []}
11465 {'Name': [], 'Address': []}
11466 {'Name': [], 'Address': []}
11467 {'Name': [], 'Address': []}
11468 {'Name': [], 'Address': []}
11469 {'Name': [], 'Address': ['87 William']}
11470 {'Name': [], 'Address': []}
11471 {'Name': [], 'Add

12444 {'Name': [], 'Address': ['18 Lawrence']}
12445 {'Name': [], 'Address': []}
12446 {'Name': [], 'Address': ['56 South']}
12447 {'Name': [], 'Address': []}
12448 {'Name': [], 'Address': []}
12449 {'Name': [], 'Address': []}
12450 {'Name': [], 'Address': []}
12451 {'Name': [], 'Address': []}
12452 {'Name': [], 'Address': []}
12453 {'Name': [], 'Address': []}
12454 {'Name': [], 'Address': []}
12455 {'Name': [], 'Address': ['Henry op Amity']}
12456 {'Name': [], 'Address': []}
12457 {'Name': [], 'Address': []}
12458 {'Name': [], 'Address': []}
12459 {'Name': [], 'Address': []}
12460 {'Name': [], 'Address': ['S3 John']}
12461 {'Name': [], 'Address': []}
12462 {'Name': [], 'Address': []}
12463 {'Name': [], 'Address': []}
12464 {'Name': [], 'Address': ['176 Front']}
12465 {'Name': [], 'Address': []}
12466 {'Name': [], 'Address': []}
12467 {'Name': [], 'Address': []}
12468 {'Name': [], 'Address': []}
12469 {'Name': [], 'Address': []}
12470 {'Name': [], 'Address': []}
12471 {'Name': [], 'Add

13443 {'Name': [], 'Address': []}
13444 {'Name': [], 'Address': ['mer']}
13445 {'Name': [], 'Address': ['3 Front']}
13446 {'Name': [], 'Address': []}
13447 {'Name': [], 'Address': ['18 Dutch st']}
13448 {'Name': [], 'Address': []}
13449 {'Name': [], 'Address': []}
13450 {'Name': [], 'Address': ['City bank']}
13451 {'Name': [], 'Address': []}
13452 {'Name': [], 'Address': []}
13453 {'Name': [], 'Address': []}
13454 {'Name': [], 'Address': []}
13455 {'Name': [], 'Address': ['76 Broad']}
13456 {'Name': [], 'Address': []}
13457 {'Name': [], 'Address': []}
13458 {'Name': [], 'Address': ['119 Maiden lane']}
13459 {'Name': [], 'Address': []}
13460 {'Name': [], 'Address': []}
13461 {'Name': [], 'Address': []}
13462 {'Name': [], 'Address': []}
13463 {'Name': [], 'Address': ['51 Exchange pl']}
13464 {'Name': [], 'Address': []}
13465 {'Name': [], 'Address': []}
13466 {'Name': [], 'Address': []}
13467 {'Name': [], 'Address': []}
13468 {'Name': [], 'Address': []}
13469 {'Name': [], 'Address': ['233

14443 {'Name': [], 'Address': []}
14444 {'Name': [], 'Address': []}
14445 {'Name': [], 'Address': []}
14446 {'Name': [], 'Address': []}
14447 {'Name': [], 'Address': []}
14448 {'Name': [], 'Address': ['Catherine ferry']}
14449 {'Name': [], 'Address': []}
14450 {'Name': [], 'Address': []}
14451 {'Name': [], 'Address': []}
14452 {'Name': [], 'Address': []}
14453 {'Name': [], 'Address': []}
14454 {'Name': [], 'Address': []}
14455 {'Name': [], 'Address': []}
14456 {'Name': [], 'Address': []}
14457 {'Name': [], 'Address': []}
14458 {'Name': [], 'Address': []}
14459 {'Name': [], 'Address': []}
14460 {'Name': [], 'Address': ['10 Front']}
14461 {'Name': [], 'Address': []}
14462 {'Name': [], 'Address': []}
14463 {'Name': [], 'Address': []}
14464 {'Name': [], 'Address': []}
14465 {'Name': [], 'Address': []}
14466 {'Name': [], 'Address': []}
14467 {'Name': [], 'Address': []}
14468 {'Name': [], 'Address': []}
14469 {'Name': [], 'Address': []}
14470 {'Name': [], 'Address': []}
14471 {'Name': [], 'A

15283 {'Name': [], 'Address': []}
15284 {'Name': [], 'Address': ['200 Fulton']}
15285 {'Name': [], 'Address': []}
15286 {'Name': [], 'Address': []}
15287 {'Name': [], 'Address': []}
15288 {'Name': [], 'Address': []}
15289 {'Name': [], 'Address': []}
15290 {'Name': [], 'Address': ['by']}
15291 {'Name': [], 'Address': []}
15292 {'Name': [], 'Address': []}
15293 {'Name': [], 'Address': []}
15294 {'Name': [], 'Address': []}
15295 {'Name': [], 'Address': []}
15296 {'Name': [], 'Address': ['25 Depeyster st']}
15297 {'Name': [], 'Address': []}
15298 {'Name': [], 'Address': []}
15299 {'Name': [], 'Address': []}
15300 {'Name': [], 'Address': ['42 Wall']}
15301 {'Name': [], 'Address': []}
15302 {'Name': [], 'Address': []}
15303 {'Name': [], 'Address': []}
15304 {'Name': [], 'Address': []}
15305 {'Name': [], 'Address': ['53 Wall']}
15306 {'Name': [], 'Address': []}
15307 {'Name': [], 'Address': []}
15308 {'Name': [], 'Address': ['138Maiden Lane']}
15309 {'Name': [], 'Address': []}
15310 {'Name': 

In [16]:
len(final_output)

15676

In [17]:
#save final output
#import json
#with open('/Users/prajwal/Desktop/Columbia/C4SR/doubt/doubt5 filter11-2.json', 'w') as f:
    #json.dump(final_output,f)
#f.close()

In [18]:
import pandas as pd
final_output_df = pd.DataFrame(final_output).T

# Save pandas dataframe as pickle object
#with open("/Users/prajwal/Desktop/Columbia/C4SR/doubt/doubt5 filter11-2.pkl", 'wb') as f:
    #pickle.dump(final_output_df, f)

# Save to csv
#final_output_df.to_csv('/Users/prajwal/Desktop/Columbia/C4SR/doubt/doubt6-filter11.csv')
#final_output_df.to_csv('/Users/prajwal/Desktop/Columbia/C4SR/mn_1850/doubt-mn1850-v7 new.csv')
final_output_df.to_csv('/Users/prajwal/Desktop/Columbia/C4SR/tesseract lib/bk 1850/address 2/data2/bk-1850-3-address2.csv')

In [19]:
len(final_output_df)

15676